In [1]:
%%capture
!pip install nltk
import nltk
nltk.download('all')
nltk.download('wordnet')
nltk.download('stopwords')

In [1]:
import pandas as pd
import nltk
import re
from nltk.stem import WordNetLemmatizer
from pickle import dump
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model

2022-03-26 01:44:34.962235: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-26 01:44:34.962311: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from google.cloud import storage

BUCKET_NAME = 'quotes_for_posts_783'

BUCKET_TRAIN_DATA_PATH = 'raw_data/image_dataset/image_dataset'
BUCKET_TRAIN_DATA_PATH_1= 'raw_data/image_dataset/df_images_0.csv'
glove_path = 'raw_data/image_dataset/glove.6B.300d.txt'

In [3]:
def download_blob(bucket_name, source_blob_name, destination_file_name):
    """Downloads a blob from the bucket."""
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)

In [4]:
def extract_features(directory, df_im_0):
    
    model = VGG16()
    
    #remove last layer
    model.layers.pop()
    
    model = Model(inputs = model.inputs , outputs = model.layers[-1].output)
    
    print(model.summary())
    
    features = dict()
    
    for name in listdir(directory):
        
        # load and image
        
    
        images = list(df_im_0)

        for i in images:
            
            filename = directory + '/' + name

            #imagePath =f'{directory} /{i}.jpg'

            #download_blob(BUCKET_NAME, imagePath, 'temp.jpg')

            #image = load_img('temp.jpg' , target_size=(224 , 224))

            image = load_img(filename , target_size=(224 , 224))

            image = img_to_array(image)

            image = image.reshape((1 , image.shape[0] , image.shape[1] ,image.shape[2]))

            image = preprocess_input(image)

            feature = model.predict(image , verbose = 0)

            # get image id
            image_id = name.split(".")[0]

            # store features
            features[image_id] = feature

            print(name)

    return features

In [5]:
df_im_0 = pd.read_csv(f'gs://{BUCKET_NAME}/{BUCKET_TRAIN_DATA_PATH_1}')
df_im_0 = df_im_0.head(100)

In [6]:
df_im_0

,image_name,comments
0,3876652419,A guy in a black shirt is looking to catch a ...
1,3876652419,Young man in blue shirt throwing a watermelon...
2,3876652419,Three young boys are engaged in a game of do ...
3,3876652419,Three teenagers unloading a truck with waterm...
4,3876652419,Three young boys tossing a watermelon .
...,...,...
95,2778313163,A woman in a blue t-shirt and jeans is sweepi...
96,2778313163,A young woman with a black shirt and jeans sw...
97,2778313163,A young woman sweeping with a straw broom .
98,2778313163,a woman sweeps the foundation of a house .


In [8]:
df_im_0['image_name'] = df_im_0['image_name'].astype('str') + '.jpg'

In [9]:
pd.unique(df_im_0['image_name'])

array(['3876652419.jpg', '2147870343.jpg', '3071144200.jpg',
       '2521062020.jpg', '4477602852.jpg', '5590283636.jpg',
       '96113759.jpg', '3533660418.jpg', '3041642723.jpg',
       '3260768565.jpg', '4890769146.jpg', '3264832994.jpg',
       '2289751916.jpg', '2987576188.jpg', '2469827608.jpg',
       '3078402009.jpg', '2174648405.jpg', '289387198.jpg',
       '221063801.jpg', '2778313163.jpg'], dtype=object)

In [10]:
from os import listdir
#directory = 'gs://{BUCKET_NAME}/{BUCKET_TRAIN_DATA_PATH}'
directory = '../../raw_data/image_dataset/small_data'
features = extract_features(directory, df_im_0)
print('extracted features :',len(features))
dump(features , open('features.pkl' , 'wb'))

2022-03-26 11:31:07.326234: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-03-26 11:31:07.521988: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-26 11:31:07.522208: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-03-26 11:31:07.522290: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-03-26 11:31:07.522364: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [11]:
lemma = WordNetLemmatizer()

def load_decriptions(doc):
    mapping = dict()
    
    for i in range(len(doc)):
        image_id = doc['image_name'][i]
        image_desc = doc['comments'][i]
        
        if image_id not in mapping:
            mapping[image_id] = list()
        
        mapping[image_id].append(image_desc)
        
    return mapping

In [12]:
def clean_text(desc):
    
    # clean punctuation
    desc = re.sub(r'[^\w\s]' ,'', desc)
    
    # tokenize the words
    desc = desc.split()
    
    # convert to lower case
    desc = [token.lower() for token in desc]
    
    # lemmatization
    desc = [lemma.lemmatize(token) for token in desc]
    
    # remove numerical values
    desc = [token for token in desc if token.isalpha()]
    
    # join whole token
    desc = ' '.join(desc)
    
    return desc

In [13]:
# convert loaded descriptions into vocablury
def to_vocabluary(descriptions):
    all_desc = set()
    
    for key in descriptions.keys():
        [all_desc.update(d.split()) for d in descriptions[key]]
        
    return all_desc

def save_descriptions(descriptions , filename):
    lines = list()
    
    for key , desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key +' '+ desc)
            
    data = '\n'.join(lines)
    file = open(filename , 'w')
    file.write(data)
    file.close()

In [32]:
%%capture
download_blob(BUCKET_NAME, glove_path, 'temp.txt')
glove = open('temp.txt', encoding="utf-8")

In [14]:
df_im_0['comments'] = df_im_0['comments'].apply(lambda x : clean_text(str(x)))

In [15]:
df_im_0.head()

,image_name,comments
0,3876652419.jpg,a guy in a black shirt is looking to catch a w...
1,3876652419.jpg,young man in blue shirt throwing a watermelon ...
2,3876652419.jpg,three young boy are engaged in a game of do nt...
3,3876652419.jpg,three teenager unloading a truck with watermelon
4,3876652419.jpg,three young boy tossing a watermelon


In [16]:
desc_map = load_decriptions(df_im_0)

In [17]:
vocabulary = to_vocabluary(desc_map)

In [18]:
len(vocabulary)

349

In [19]:
save_descriptions(desc_map , 'descriptions.txt')

In [20]:
from pickle import load

In [21]:
# loading the doc
def load_doc(filename):
    file = open(filename , 'r')
    text = file.read()
    file.close()
    return text

# this function is used for to get train image description from our dataset

def load_clean_descriptions(filename , dataset):
    doc = load_doc(filename)
    descriptions = dict()
    
    for line in doc.split('\n'):
        tokens = line.split()
        image_name , image_desc = tokens[0] , tokens[1:]
        
        if image_name in dataset:
            
            if image_name not in descriptions:
                descriptions[image_name] = list()
            
            # we add two tage at start and at end of the descitpion to identify to start and 
            # end of desc.
            desc = 'startseq '+ ' '.join(image_desc)+ ' endseq'
            descriptions[image_name].append(desc)
            
    return descriptions


# laod photo features
def load_photo_features(filename , dataset):
    all_features = load(open(filename,'rb'))
    features = {k+'.jpg' : all_features[k] for k in dataset}
    return features

In [22]:
train = set(df_im_0['image_name'])
print('len of train image',len(train))

len of train image 20


In [23]:
train_descriptions = load_clean_descriptions('descriptions.txt' , train)
print('len of train descriptions' , len(train_descriptions))

len of train descriptions 20


In [24]:
train = pd.DataFrame(train)
train2 = train[0].apply(lambda x : x.replace('.jpg' , ''))
train_features = load_photo_features('features.pkl' , train2)
print('photos train :',len(train_features))

photos train : 20


In [25]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [26]:

def to_lines(descriptions):
    all_desc = list()
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
        
    return all_desc

# fit tokenizer on descriptions
def create_tokenizer(descriptions):
    lines = to_lines(descriptions)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [27]:
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('vocab size' , vocab_size)

vocab size 352


In [30]:
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

In [31]:
# calculate the length with most words
def max_length(descriptions):
    lines = to_lines(descriptions)
    return max([len(line.split())for line in lines])
    
# create sequences of images,input sequences and output sequences
def create_sequences(tokenizer , max_length , desc_list , photo):
    X1 , X2 , y = list() , list() , list()
    
    for desc in desc_list:
        # convert words to number value
        seq = tokenizer.texts_to_sequences([desc])[0]
        
        for i in range(1, len(seq)):
            
            in_seq , output_seq = seq[:i] , seq[i]
            in_seq = pad_sequences([in_seq] , maxlen = max_length)[0]
            output_seq = to_categorical([output_seq] , num_classes = vocab_size)[0]
            
            X1.append(photo)
            X2.append(in_seq)
            y.append(output_seq)
            
    return np.array(X1) , np.array(X2) , np.array(y)

In [32]:
from keras.models import Model
from keras.layers import Input , Dense , LSTM , Embedding , Dropout
from keras.layers.merge import add
from keras.callbacks import EarlyStopping

In [33]:
def define_Model(vocab_size , max_length):
    
    # feature extractor model
    inputs1 = Input(shape=(4096 , ))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(512 , activation='relu')(fe1)
    fe3 = Dense(256 , activation = 'relu')(fe2)
    
    # sequence model
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size,512,mask_zero=True )(inputs2) # mask_zero = ignore padding
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(512 , return_sequences=True)(se2)
    se4 = Dropout(0.5)(se3)
    se5 = LSTM(256)(se4)
    
    
    #decoder Model
    decoder1 = add([fe3 , se5])
    decoder2 = Dense(256 , activation='relu')(decoder1)
    decoder3 = Dense(512 , activation='relu')(decoder2)
    outputs = Dense(vocab_size , activation='softmax')(decoder3)
    
    # combine both image and text
    model = Model([inputs1 , inputs2] , outputs)
    model.compile(loss='categorical_crossentropy' , optimizer = 'adam')
    
    # summary
    print(model.summary())
    
    return model

In [34]:
def data_generator(descriptions , photos , tokenizer , max_length):
    while 1:
        for key , desc_list in descriptions.items():
            photo = photos[key][0]
            in_img , in_seq , out_seq = create_sequences(tokenizer , max_length , desc_list , photo)
            
            yield[[in_img , in_seq] , out_seq]

In [35]:

print('len :', len(train))
print('Descriptions :',len(train_descriptions))
print('photos train :',len(train_features))
print('Vocabulary size :',vocab_size)
max_len = max_length(train_descriptions)
print('Description max length :', max_len)



len : 20
Descriptions : 20
photos train : 20
Vocabulary size : 352
Description max length : 26


In [36]:
from keras.models import load_model

In [37]:
model = define_Model(vocab_size , max_len)
epochs = 5
steps = len(train_descriptions)

for i in range(epochs):
    generator = data_generator(train_descriptions , train_features , tokenizer , max_len)
    
    model.fit_generator(generator , epochs = 1 , steps_per_epoch = steps , verbose = 1)
    
    model.save('model_'+ str(i+1) + '.h5')

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 26)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 26, 512)      180224      ['input_3[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, 4096)]       0           []                               
                                                                                                  
 dropout_1 (Dropout)            (None, 26, 512)      0           ['embedding[0][0]']              
                                                                                            

/tmp/ipykernel_1508/2303626749.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator , epochs = 1 , steps_per_epoch = steps , verbose = 1)


InvalidArgumentError: Graph execution error:

Detected at node 'model_1/dense/Relu' defined at (most recent call last):
    File "/home/mohanakrishnan/.pyenv/versions/3.8.12/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/mohanakrishnan/.pyenv/versions/3.8.12/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/mohanakrishnan/.pyenv/versions/3.8.12/envs/quotes_for_posts_783/lib/python3.8/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/home/mohanakrishnan/.pyenv/versions/3.8.12/envs/quotes_for_posts_783/lib/python3.8/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/home/mohanakrishnan/.pyenv/versions/3.8.12/envs/quotes_for_posts_783/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 677, in start
      self.io_loop.start()
    File "/home/mohanakrishnan/.pye

In [ ]:


def word_for_id(integer , tokenizer):
    for word,index in tokenizer.word_index.items():
        if index == integer:
            return word



In [38]:
def generate_desc(model , tokenizer , photo , max_length):
    
    input_text = 'startseq'
    
    for i in range(max_length):
        
        sequence = tokenizer.texts_to_sequences([input_text])[0]
        sequence = pad_sequences([sequence] , maxlen=max_length)
        
        # predict the next word
        next_word_id = model.predict([photo,sequence],verbose = 0)
        
        # get highest probality word from list of words
        next_word_id = np.argmax(next_word_id)
        
        # get word from id
        word = word_for_id(next_word_id , tokenizer)
        
        if word is None:
            break
            
        # update input text
        input_text += ' '+ word
        
        if word == 'endseq':
            break
            
    return input_text

In [39]:
from nltk.translate.bleu_score import corpus_bleu

In [40]:
def evaluate_model(model , desciptions , photos , tokenizer , max_length):
    actual , predicted = list() , list()
    
    for key , desc_list in desciptions.items():
        generated_desc = generate_desc(model , tokenizer , photos[key] , max_length)
        
        references = [d.split() for d in desc_list]
        actual.append(references)
        predicted.append(generated_desc.split())
    
    print('Bleu_Score -1 = %f'%corpus_bleu(actual , predicted , weights=(1,0,0,0)))
    
    print('Bleu_Score -2 = %f'%corpus_bleu(actual , predicted , weights=(0.5,0.5,0,0)))
    
    print('Bleu_Score -3 = %f'%corpus_bleu(actual , predicted , weights=(0.33,0.33,0.33,0))) 
    
    print('Bleu_Score -4 = %f'%corpus_bleu(actual , predicted , weights=(0.25,0.25,0.25,0.25)))